## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clarence Town,AU,-32.5833,151.7833,69.35,48,0,12.15,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,33.46,86,20,14.97,few clouds
2,2,Chokurdakh,RU,70.6333,147.9167,56.43,40,56,9.98,broken clouds
3,3,Maragogi,BR,-9.0122,-35.2225,72.79,88,28,10.07,scattered clouds
4,4,Qaanaaq,GL,77.4840,-69.3632,36.86,74,100,2.82,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clarence Town,AU,-32.5833,151.7833,69.35,48,0,12.15,clear sky
3,3,Maragogi,BR,-9.0122,-35.2225,72.79,88,28,10.07,scattered clouds
5,5,Itamaraca,BR,-7.7478,-34.8256,72.28,86,60,11.92,broken clouds
8,8,Konevo,RU,62.1215,39.3192,62.15,67,92,7.23,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,71.80,74,30,7.38,scattered clouds
13,13,Albany,US,42.6001,-73.9662,69.39,94,0,3.76,clear sky
14,14,Geraldton,AU,-28.7667,114.6000,67.12,72,40,16.11,scattered clouds
15,15,Touros,BR,-5.1989,-35.4608,72.50,91,10,9.82,clear sky
22,22,Janauba,BR,-15.8025,-43.3089,61.63,58,0,7.05,clear sky
30,30,Puerto Ayora,EC,-0.7393,-90.3518,71.55,89,7,4.00,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                240
City                   240
Country                239
Lat                    240
Lng                    240
Max Temp               240
Humidity               240
Cloudiness             240
Wind Speed             240
Current Description    240
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clarence Town,AU,-32.5833,151.7833,69.35,48,0,12.15,clear sky
3,3,Maragogi,BR,-9.0122,-35.2225,72.79,88,28,10.07,scattered clouds
5,5,Itamaraca,BR,-7.7478,-34.8256,72.28,86,60,11.92,broken clouds
8,8,Konevo,RU,62.1215,39.3192,62.15,67,92,7.23,overcast clouds
10,10,Rikitea,PF,-23.1203,-134.9692,71.80,74,30,7.38,scattered clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Clarence Town,AU,69.35,clear sky,-32.5833,151.7833,
3,Maragogi,BR,72.79,scattered clouds,-9.0122,-35.2225,
5,Itamaraca,BR,72.28,broken clouds,-7.7478,-34.8256,
8,Konevo,RU,62.15,overcast clouds,62.1215,39.3192,
10,Rikitea,PF,71.80,scattered clouds,-23.1203,-134.9692,
13,Albany,US,69.39,clear sky,42.6001,-73.9662,
14,Geraldton,AU,67.12,scattered clouds,-28.7667,114.6000,
15,Touros,BR,72.50,clear sky,-5.1989,-35.4608,
22,Janauba,BR,61.63,clear sky,-15.8025,-43.3089,
30,Puerto Ayora,EC,71.55,clear sky,-0.7393,-90.3518,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Clarence Town,AU,69.35,clear sky,-32.5833,151.7833,River Downs Cottage
3,Maragogi,BR,72.79,scattered clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
5,Itamaraca,BR,72.28,broken clouds,-7.7478,-34.8256,Special K's Bed & Breakfast
8,Konevo,RU,62.15,overcast clouds,62.1215,39.3192,
10,Rikitea,PF,71.80,scattered clouds,-23.1203,-134.9692,People ThankYou
13,Albany,US,69.39,clear sky,42.6001,-73.9662,
14,Geraldton,AU,67.12,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
15,Touros,BR,72.50,clear sky,-5.1989,-35.4608,INN NEW HORIZON
22,Janauba,BR,61.63,clear sky,-15.8025,-43.3089,Ouro Norte Palace Hotel
30,Puerto Ayora,EC,71.55,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] !=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Clarence Town,AU,69.35,clear sky,-32.5833,151.7833,River Downs Cottage
3,Maragogi,BR,72.79,scattered clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
5,Itamaraca,BR,72.28,broken clouds,-7.7478,-34.8256,Special K's Bed & Breakfast
10,Rikitea,PF,71.80,scattered clouds,-23.1203,-134.9692,People ThankYou
14,Geraldton,AU,67.12,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
15,Touros,BR,72.50,clear sky,-5.1989,-35.4608,INN NEW HORIZON
22,Janauba,BR,61.63,clear sky,-15.8025,-43.3089,Ouro Norte Palace Hotel
30,Puerto Ayora,EC,71.55,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel
32,Esperance,AU,62.44,clear sky,-33.8667,121.9000,Hospitality Esperance
35,Lasa,IT,62.02,moderate rain,46.6166,10.7002,Pension Feldgärtenhof


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather with Max Temp</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))